Вы успешно создали полноценную торговую стратегию на основе модели машинного обучения и обеспечили фиксацию и сравнение метрик полученных моделей.

Вы решаете построить более сложные модели машинного обучения и хотите использовать нейросетевые модели, в том числе для обработки временных рядов и глубокие нейронные сети. Здесь вам могут помочь методы рекуррентных сетей и современные трансформерные архитектуры.

Поговорив с коллегами, вы понимаете, что самостоятельно построить и обучить действительно сложные архитектуры будет сложно и решаете воспользоваться предобученными свободно распространяемыми моделями.

На основании вышесказанного вам необходимо построить несколько моделей на основе нейронных сетей, позволяющих прогнозировать оптимальное торговое действие.

На основе представленной информации, вам предлагается:

Создать модель (торговую стратегию) на основе нейронных сетей для прогнозирования оптимального торгового действия. Можно использовать, как самостоятельно обученные архитектуры, так и использовать предобученные сети или фреймворки.
Провести тестирование разработанной стратегии на валидационном датасете.
Зафиксировать метрики модели для дальнейшего сравнения экспериментов.
Сформировать дашборд, показывающий эффективность различных торговых
стратегий.